# Détection d'anomalies en temps réel sur les logs

Ce notebook permet d'appliquer un modèle pré-entraîné de détection d'anomalies sur de nouveaux logs système. Il utilise :
- Un modèle Isolation Forest pré-entraîné
- Des encodeurs de caractéristiques sauvegardés
- Un pipeline de prétraitement des données

## 1. Import des bibliothèques nécessaires

In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import joblib

## 2. Fonction de chargement et préparation des données

Cette fonction :
- Charge les données avec les types appropriés
- Convertit les dates en timestamps
- Optimise la mémoire en utilisant des types catégoriels

In [17]:
def load_and_prepare_data(file_path):
    """
    Charge et prépare les données pour la détection
    """
    dtypes = {
        'Hostname': 'category',
        'Process': 'category',
        'IdProcess': 'str',
        'Message': 'str'
    }
    parse_dates = ['Date']

    # Charger les données
    df = pd.read_csv(file_path, dtype=dtypes, parse_dates=parse_dates)

    # Convertir datetime en timestamp
    df['Date'] = df['Date'].astype('int64') // 10**9

    return df

# Conversion du modèle au format .joblib

* Cette fonction convertie le modèle en .plk au format .joblib.

In [18]:
import pickle
from joblib import dump

# Charger le modèle au format .pkl
with open("model_isolation_forest.pkl", "rb") as file:
    model = pickle.load(file)

# Sauvegarder le modèle au format joblib
dump(model, "isolation_forest_model.joblib")

print("Modèle converti et sauvegardé au format joblib avec succès.")


Modèle converti et sauvegardé au format joblib avec succès.


## 3. Fonction de détection des anomalies

Cette fonction :
- Charge le modèle et les encodeurs pré-entraînés
- Encode les nouvelles données de la même manière que les données d'entraînement
- Applique le modèle pour détecter les anomalies

In [19]:
def detect_anomalies(df, model_path='isolation_forest_model.joblib', encoders_path='feature_encoders.joblib'):
    """
    Detect anomalies in a new dataset
    """
    # Load the model
    saved_model = joblib.load(model_path)
    if isinstance(saved_model, dict):
        model = saved_model.get('model')
        scaler = saved_model.get('scaler')
    else:
        model = saved_model
        scaler = None  # Scaler may not exist in this case

    # Load the encoders
    encoders = joblib.load(encoders_path)

    # Encode categorical features
    df_encoded = df.copy()
    for column, encoder in encoders.items():
        df_encoded[column] = df_encoded[column].map(
            lambda x: -1 if x not in encoder.classes_ else encoder.transform([x])[0]
        )

    # Normalize data if scaler exists
    if scaler:
        X_scaled = scaler.transform(df_encoded)
    else:
        X_scaled = df_encoded  # Use raw data if no scaler is available

    # Predict anomalies
    predictions = model.predict(X_scaled)

    # Convert predictions (-1: anomaly, 1: normal)
    anomalies = np.where(predictions == -1, 1, 0)

    # Add predictions to the original DataFrame
    df['anomaly'] = anomalies

    return df


In [20]:
saved_model = joblib.load("isolation_forest_model.joblib")
print(saved_model)

['Date_2024-12-11T16:37:38.628767+01:00'
 'Date_2024-12-11T16:37:38.629082+01:00'
 'Date_2024-12-11T16:37:38.630478+01:00' ...
 'Message_xdg-desktop-portal-gnome.service: Main process exited, code=exited, status=1/FAILURE'
 "Message_xdg-desktop-portal-gtk.service: Failed with result 'exit-code'."
 'Message_xdg-desktop-portal-gtk.service: Main process exited, code=exited, status=1/FAILURE']


## 4. Exécution de la détection

Cette section exécute le pipeline complet de détection d'anomalies sur les nouvelles données.

In [ ]:
# Chemin vers votre nouveau fichier CSV
new_data_path = "corrupted_logs.csv"

print("Chargement des données...")
df = load_and_prepare_data(new_data_path)

print("Détection des anomalies...")
results = detect_anomalies(df)

# Afficher un résumé
n_anomalies = results['anomaly'].sum()
print(f"\nNombre total d'anomalies détectées : {n_anomalies}")
print(f"Pourcentage d'anomalies : {(n_anomalies / len(results)) * 100:.2f}%")

# Sauvegarder les résultats
output_file = "resultats_anomalies.csv"
results.to_csv(output_file, index=False)
print(f"\nRésultats sauvegardés dans : {output_file}")

# Afficher quelques exemples d'anomalies
print("\nExemples d'anomalies détectées :")
print(results[results['anomaly'] == 1].head())

Chargement des données...
Détection des anomalies...


## 5. Analyse des résultats

Pour approfondir l'analyse des anomalies détectées, vous pouvez :
1. Examiner la distribution temporelle des anomalies
2. Analyser les patterns par host ou par processus
3. Identifier les messages les plus fréquents dans les anomalies

Voici quelques visualisations utiles :

In [ ]:
# Distribution des anomalies par host
anomalies_by_host = results[results['anomaly'] == 1]['Hostname'].value_counts()
print("Distribution des anomalies par host :")
print(anomalies_by_host.head())

# Distribution des anomalies par processus
anomalies_by_process = results[results['anomaly'] == 1]['Process'].value_counts()
print("\nDistribution des anomalies par processus :")
print(anomalies_by_process.head())